Test a multitude of functions and methods from interactions.py using the "fiction" system. To check, run this notebook and visually check the resulting system in MD visualization software. Methods/functions tested:

### interactions.py
- Reaction
    - create_product_molecules()
    - calculate_product_positions()
- update_molecules_list_IDs()
- find_overlaps()
- remove_overlaps()
- remove_overlaps_from_molecules_list()
- update_molecules_list_with_reaction()
- get_interactions_lists_from_molecules_list()

In [10]:
import numpy as np
from copy import deepcopy
from hybrid_mdmc import utility
from hybrid_mdmc.system import SystemData
from hybrid_mdmc.interactions import *
from hybrid_mdmc.filehandlers import write_lammps_data

In [11]:
reactant1 = Molecule(
        ID=1,
        kind="A2",
        atoms=[
            Atom(ID=1, kind=1, lammps_type=1, element="A", mass=1.0000, x=0.0, y=0.0, z=0.0, charge=0.0),
            Atom(ID=2, kind=1, lammps_type=1, element="A", mass=1.0000, x=2.0, y=0.0, z=0.0, charge=0.0),
        ],
        bonds=[
            IntraMode(ID=1, kind=1, atom_IDs=[1,2])
        ],
    )
reactant2 = Molecule(
        ID=1,
        kind="AB2C",
        atoms=[
            Atom(ID=3, kind=1, lammps_type=1, element="A", mass=1.0000, x=0.0, y=0.0, z=0.0, charge=0.1),
            Atom(ID=4, kind=2, lammps_type=2, element="B", mass=2.0000, x=1.0, y=1.0, z=0.0, charge=-0.1),
            Atom(ID=5, kind=2, lammps_type=2, element="B", mass=2.0000, x=3.0, y=1.0, z=0.0, charge=0.0),
            Atom(ID=6, kind=3, lammps_type=3, element="C", mass=1.5000, x=4.0, y=2.0, z=0.0, charge=0.0),
        ],
        bonds=[
            IntraMode(ID=1, kind=2, atom_IDs=[3,4]),
            IntraMode(ID=2, kind=3, atom_IDs=[4,5]),
            IntraMode(ID=3, kind=4, atom_IDs=[5,6]),
        ],
        angles=[
            IntraMode(ID=1, kind=1, atom_IDs=[3,4,5]),
            IntraMode(ID=2, kind=2, atom_IDs=[4,5,6]),
        ],
        dihedrals=[
            IntraMode(ID=1, kind=1, atom_IDs=[3,4,5,6]),
        ],
    )
product1 = Molecule(
        ID=1,
        kind="A2B2",
        atoms=[
            Atom(ID=4, kind=2, lammps_type=2, element="B", mass=2.0000, x=0.0, y=0.0, z=0.0, charge=-0.1),
            Atom(ID=5, kind=2, lammps_type=2, element="B", mass=2.0000, x=1.0, y=-1.0, z=0.0, charge=0.0),
            Atom(ID=3, kind=1, lammps_type=1, element="A", mass=1.0000, x=-1.0, y=-1.0, z=0.0, charge=0.05),
            Atom(ID=1, kind=1, lammps_type=1, element="A", mass=1.0000, x=0.0, y=2.0, z=0.0, charge=0.05),
        ],
        bonds=[
            IntraMode(ID=1, kind=3, atom_IDs=[4,5]),
            IntraMode(ID=2, kind=4, atom_IDs=[1,4]),
            IntraMode(ID=3, kind=4, atom_IDs=[3,4]),
        ],
        angles=[
            IntraMode(ID=1, kind=3, atom_IDs=[1,4,5]),
            IntraMode(ID=2, kind=3, atom_IDs=[3,4,5]),
            IntraMode(ID=3, kind=4, atom_IDs=[1,4,3]),
        ],
        impropers=[
            IntraMode(ID=1, kind=1, atom_IDs=[4,1,3,5]),
        ]
    )
product2 = Molecule(
        ID=1,
        kind="AC",
        atoms=[
            Atom(ID=2, kind=1, lammps_type=1, element="A", mass=1.0000, x=0.0, y=0.0, z=0.0, charge=0.0),
            Atom(ID=6, kind=3, lammps_type=3, element="C", mass=1.5000, x=2.0, y=0.0, z=0.0, charge=0.0),
        ],
        bonds=[
            IntraMode(ID=1, kind=5, atom_IDs=[2,6]),
        ],
    )
reaction1 = Reaction(
    ID=1,
    kind=1,
    reactant_molecules=[reactant1, reactant2],
    product_molecules=[product1,product2],
    Ea=4.0,
    Ea_units="kcal/mol",
    A=1e12,
    A_units="1/s",
    b=1,
)

In [12]:
A2 = deepcopy(reactant1)
AB2C = deepcopy(reactant2)
A2B2 = deepcopy(product1)
AC = deepcopy(product2)

A2.clean_IDs(new_ID=1)
AB2C.clean_IDs(new_ID=1)
A2B2.clean_IDs(new_ID=1)
AC.clean_IDs(new_ID=1)

system_data = SystemData("fiction", "fiction")
system_data.species = [A2, AB2C, A2B2, AC]
system_data.reactions = [reaction1]
system_data.write_json()

In [13]:
system_data = SystemData("fiction", "fiction")
system_data.read_json()
A2, AB2C, A2B2, AC = system_data.species

molecules_list = []

new = deepcopy(A2)
new.clean_IDs(new_ID=1)
new.calculate_cog()
new.translate(np.array([-5,-5,-5]))
molecules_list.append(new)

new = deepcopy(A2)
new.clean_IDs(new_ID=2)
new.adjust_atom_IDs(offset=2)
new.adjust_intramode_IDs(offset=1, mode="bonds")
new.calculate_cog()
new.translate(np.array([-5,-5,5]))
molecules_list.append(new)

new = deepcopy(AB2C)
new.clean_IDs(new_ID=3)
new.adjust_atom_IDs(offset=4)
new.adjust_intramode_IDs(offset=2, mode="bonds")
new.adjust_intramode_IDs(offset=0, mode="angles")
new.calculate_cog()
new.translate(np.array([-5,5,-5]))
molecules_list.append(new)

new = deepcopy(AB2C)
new.clean_IDs(new_ID=4)
new.adjust_atom_IDs(offset=8)
new.adjust_intramode_IDs(offset=5, mode="bonds")
new.adjust_intramode_IDs(offset=2, mode="angles")
new.adjust_intramode_IDs(offset=1, mode="dihedrals")
new.adjust_intramode_IDs(offset=0, mode="impropers")
new.calculate_cog()
new.translate(np.array([-5,5,5]))
molecules_list.append(new)

new = deepcopy(A2B2)
new.clean_IDs(new_ID=5)
new.adjust_atom_IDs(offset=12)
new.adjust_intramode_IDs(offset=8, mode="bonds")
new.adjust_intramode_IDs(offset=4, mode="angles")
new.adjust_intramode_IDs(offset=2, mode="dihedrals")
new.adjust_intramode_IDs(offset=0, mode="impropers")
new.calculate_cog()
new.translate(np.array([5,-5,-5]))
molecules_list.append(new)

new = deepcopy(A2B2)
new.clean_IDs(new_ID=6)
new.adjust_atom_IDs(offset=16)
new.adjust_intramode_IDs(offset=11, mode="bonds")
new.adjust_intramode_IDs(offset=7, mode="angles")
new.adjust_intramode_IDs(offset=2, mode="dihedrals")
new.adjust_intramode_IDs(offset=1, mode="impropers")
new.calculate_cog()
new.translate(np.array([5,-5,5]))
molecules_list.append(new)

new = deepcopy(AC)
new.clean_IDs(new_ID=7)
new.adjust_atom_IDs(offset=20)
new.adjust_intramode_IDs(offset=14, mode="bonds")
new.adjust_intramode_IDs(offset=10, mode="angles")
new.adjust_intramode_IDs(offset=2, mode="dihedrals")
new.adjust_intramode_IDs(offset=2, mode="impropers")
new.calculate_cog()
new.translate(np.array([5,5,-5]))
molecules_list.append(new)

new = deepcopy(AC)
new.clean_IDs(new_ID=8)
new.adjust_atom_IDs(offset=22)
new.adjust_intramode_IDs(offset=15, mode="bonds")
new.adjust_intramode_IDs(offset=10, mode="angles")
new.adjust_intramode_IDs(offset=2, mode="dihedrals")
new.adjust_intramode_IDs(offset=2, mode="impropers")
new.calculate_cog()
new.translate(np.array([5,5,5]))
molecules_list.append(new)

print(f"molecule IDs: {[_.ID for _ in molecules_list]}")
print(f"atoms: {utility.flatten_nested_list([[_.ID for _ in molecule.atoms] for molecule in molecules_list])}")
print(f"bonds: {utility.flatten_nested_list([[_.ID for _ in molecule.bonds] for molecule in molecules_list if molecule.bonds is not None])}")
print(f"angles: {utility.flatten_nested_list([[_.ID for _ in molecule.angles] for molecule in molecules_list if molecule.angles is not None])}")
print(f"dihedrals: {utility.flatten_nested_list([[_.ID for _ in molecule.dihedrals] for molecule in molecules_list if molecule.dihedrals is not None])}")
print(f"impropers: {utility.flatten_nested_list([[_.ID for _ in molecule.impropers] for molecule in molecules_list if molecule.impropers is not None])}")

molecule IDs: [1, 2, 3, 4, 5, 6, 7, 8]
atoms: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
bonds: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
angles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
dihedrals: [1, 2]
impropers: [1, 2]


In [14]:

atoms_list, bonds_list, angles_list, dihedrals_list, impropers_list = get_interactions_lists_from_molcules_list(molecules_list)
write_lammps_data(
    filename="fiction.in.data",
    box=[ [-10,10], [-10,10], [-10,10] ],
    atoms_list=atoms_list,
    bonds_list=bonds_list,
    angles_list=angles_list,
    dihedrals_list=dihedrals_list,
    impropers_list=impropers_list,
    wrap=False,
)


In [15]:
reaction = Reaction(
    ID=1,
    kind=1,
    reactant_molecules=[molecules_list[0], molecules_list[2]],
)
reaction.create_product_molecules(system_data.reactions[0])
molecules_list = update_molecules_list_with_reaction(molecules_list, reaction, [[-10,10], [-10,10], [-10,10]])
atoms_list, bonds_list, angles_list, dihedrals_list, impropers_list = get_interactions_lists_from_molcules_list(molecules_list)
write_lammps_data(
    filename="fiction.end.data",
    box=[ [-10,10], [-10,10], [-10,10] ],
    atoms_list=atoms_list,
    bonds_list=bonds_list,
    angles_list=angles_list,
    dihedrals_list=dihedrals_list,
    impropers_list=impropers_list,
    wrap=False,
)